In [23]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import json 
import time


driver = webdriver.Chrome(executable_path='/Users/lauratociu/Documents/chromedriver')
driver.get('https://www.moonboard.com/Account/Login')

# Is anyone tries to run this script, they must create an account with Moonboard.
# The username and password have to be inputted manually here because it has anti-robot checks at login. 
# All the data can only be accessed when logged in.
# After logging in, allow window to run the rest of the code in the background. You should not get logged out.

# Note: In next paragraphs we will find new Moonboard problems added since we last updated the database.

In [21]:
# Here we get the lost of active users. 
driver.get('https://www.moonboard.com/Account/GetActiveMoonBoardUsers')

pre = driver.find_element_by_tag_name("pre").text
data = json.loads(pre)

activeuser_ids = []
for item in data:
    activeuser_ids.append(item['UserId'])
    
print(activeuser_ids[:10])

['7498cda6-1583-4263-92a3-5c0531b0afd0', 'd3194f62-9def-4bb8-850d-97ea40ed8668', '0e537bf6-34c9-4bfa-b9c7-70c79f08d9be', '251d680f-9a88-4eb2-a9fd-2ec3dd113980', 'dad7a9e1-5213-41df-a11d-0a4f9427a862', '6FD766B9-A6B9-4C53-B7F3-21751C965E0F', '39bee971-2448-4248-8122-7ec11fd8d240', 'B1F210D7-978B-476D-812F-778D4BCB3268', 'd9ea7030-3773-4b07-9c0d-018461755fdd', '805c8aff-c33c-471a-b150-69c59223e1f4']


In [26]:
def find_new_problems(user_id):
    driver.get('https://www.moonboard.com/Account/Profile/' + user_id)
    option_el = driver.find_element_by_xpath("//option[contains(text(),'%s')]" %'MoonBoard 2016')
    option_el.click()

    time.sleep(2)  # To make sure the right Moonboard 2016 setup is selected from dropdown menu

    all_links = driver.find_elements_by_css_selector('a')
    
    # Since the user has to click to go to the Problems climbed, I am assuming that the links to Problems that appear on
    # this page are all from the category of Problems Created
    problem = []
    for link in all_links:
        if 'Problems/View' in link:
            problem.append(link)
        
# We will loop through the active users and see if anyone created a new problem on today's date.

new_problems = []
for user_id in activeuser_ids:
    new_problems += find_new_problems(user_id)


# I am not going to run this snippet since I am sure it would be a lot of computing and web scraping. Measures will 
# have to be taken to avoin getting your IP banned, but that is beyond the scope of this proof of principle script.

In [ ]:
def get_problem_name_id(problem_link):
    # This function extracts name and ID from the link in order to check if it exists in our database
    info = problem_link.split('/')
    name = info[-1]
    idd = info[-2]
    return [idd, name]

# Now we can load the existing databaase of Moonboard climb (see pre-process_database.ipynb), and for each new problem
# in new_problems, we can check if it is already in our database. If not, we add it.

